<a href="https://www.kaggle.com/code/vtrackstar/hoop-mixtape-fantasy-basketball-projections?scriptVersionId=195630672" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# Stats from BasketballReference.com Simple Projections

proj = pd.read_csv('/kaggle/input/updated2425nbaplayerstatproj/24-25 nba player stat projections_updated - sportsref_download.csv', header=0)
proj.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,...,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,-9999
0,1,Precious Achiuwa,Projected,5.6,11.3,0.8,2.7,1.9,2.9,3.8,...,2.1,1.0,1.3,1.8,3.1,13.9,0.494,0.296,0.656,achiupr01
1,2,Bam Adebayo,Projected,8.0,15.1,0.2,0.6,4.4,5.7,2.4,...,3.9,1.2,0.9,2.5,2.6,20.6,0.530,0.333,0.772,adebaba01
2,3,Ochai Agbaji,Projected,4.4,10.2,1.7,5.2,1.2,1.6,1.6,...,2.0,0.9,0.8,1.4,2.6,11.6,0.429,0.326,0.733,agbajoc01
3,4,Santi Aldama,Projected,5.6,12.2,2.2,6.3,1.7,2.4,1.7,...,2.8,1.0,1.1,1.5,2.4,15.0,0.455,0.354,0.694,aldamsa01
4,5,Nickeil Alexander-Walker,Projected,4.9,11.1,2.5,6.4,1.3,1.7,0.7,...,4.0,1.2,0.8,1.7,2.8,13.5,0.437,0.383,0.761,alexani01


In [2]:
proj.columns

Index(['Rk', 'Player', 'Type', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'FG%', '3P%', 'FT%',
       '-9999'],
      dtype='object')

In [3]:
num_rows = len(proj)
print(f"Number of rows: {num_rows}")

Number of rows: 570


In [4]:
# Create fantasy scoring

def calculate_fantasy_points(proj):
    # Yahoo Hoop Mixtape scoring rules
    proj['fantasy_points'] = (
        (proj['FGA'] * -0.45) +                # FGA (-0.45 points each)
        (proj['FG'] * 1) +                     # FGM (1 point each)
        (proj['FTA'] * -0.75) +                # FTA (-0.75 point each)
        (proj['FT'] * 1) +                     # FTM (1 point each)
        (proj['3PA'] * -1.45) +                # 3PA (-1.45 points each)
        (proj['3P'] * 3) +                     # 3PM (3 points each)
        (proj['PTS'] * 0.5) +                  # PTS (0.5 points each)
        (proj['ORB'] * 1.5) +                  # OREB (1.5 points each)
        ((proj['TRB'] - proj['ORB']) * 0.75) + # DREB (0.75 points each)
        (proj['AST'] * 2) +                    # AST (2 points each)
        (proj['STL'] * 3) +                    # STL (3 points each)
        (proj['BLK'] * 3) +                    # BLK (3 points each)
        (proj['TOV'] * -3)                     # TOV (-3 points each)
    )
    
    return proj

# Add column to table
proj = calculate_fantasy_points(proj)

# Display the first few rows of the DataFrame
print(proj)

      Rk                    Player       Type   FG   FGA   3P  3PA   FT  FTA  \
0      1          Precious Achiuwa  Projected  5.6  11.3  0.8  2.7  1.9  2.9   
1      2               Bam Adebayo  Projected  8.0  15.1  0.2  0.6  4.4  5.7   
2      3              Ochai Agbaji  Projected  4.4  10.2  1.7  5.2  1.2  1.6   
3      4              Santi Aldama  Projected  5.6  12.2  2.2  6.3  1.7  2.4   
4      5  Nickeil Alexander-Walker  Projected  4.9  11.1  2.5  6.4  1.3  1.7   
..   ...                       ...        ...  ...   ...  ...  ...  ...  ...   
565  566            Thaddeus Young  Projected  5.3   9.8  0.5  1.8  1.0  1.6   
566  567                Trae Young  Projected  8.3  19.0  2.8  7.8  6.9  7.9   
567  568            Omer Yurtseven  Projected  6.6  12.4  0.6  2.2  1.7  2.3   
568  569               Cody Zeller  Projected  4.5   9.2  0.5  1.6  2.9  4.3   
569  570               Ivica Zubac  Projected  6.3   9.9  0.1  0.3  2.6  3.6   

     ORB  ...  STL  BLK  TOV   PF   PTS

In [5]:
# Function to filter player names based on search input
def filter_player_names(search_text):
    # Filter player names that contain the search text
    filtered_players = proj[proj['Player'].str.contains(search_text, case=False, na=False)]
    return filtered_players['Player'].tolist()

# Function to update dropdown options based on search text
def update_dropdown(change):
    search_text = search_box.value
    filtered_names = filter_player_names(search_text)
    player_dropdown.options = filtered_names
    if filtered_names:
        player_dropdown.value = filtered_names[0]

# Function to handle button click
def on_button_click(b):
    selected_player = player_dropdown.value
    fantasy_points = proj.loc[proj['Player'] == selected_player, 'fantasy_points'].values
    if fantasy_points.size > 0:
        result_label.value = f"Fantasy points for {selected_player}: {fantasy_points[0]}"
    else:
        result_label.value = "Player not found."

# Create widgets
search_box = widgets.Text(
    description='Search:',
    placeholder='Type player name'
)
player_dropdown = widgets.Dropdown(
    description='Player:',
    options=[]
)
search_box.observe(update_dropdown, names='value')
result_label = widgets.Label()
button = widgets.Button(description="Get Fantasy Points")
button.on_click(on_button_click)

# Display widgets
display(search_box, player_dropdown, button, result_label)

Text(value='', description='Search:', placeholder='Type player name')

Dropdown(description='Player:', options=(), value=None)

Button(description='Get Fantasy Points', style=ButtonStyle())

Label(value='')

In [6]:
# Function to filter player names based on search input
def filter_player_names(search_text):
    search_text = search_text.strip()
    # Filter player names that contain the search text
    filtered_players = proj_multiple_players[proj_multiple_players['Player'].str.contains(search_text, case=False, na=False)]
    return filtered_players['Player'].tolist()

# Function to update dropdown options based on search text for each input
def update_dropdown(change, i):
    search_text = player_text_boxes[i].value
    filtered_names = filter_player_names(search_text)
    if filtered_names:
        player_dropdowns[i].options = filtered_names
        player_dropdowns[i].value = filtered_names[0]
    else:
        player_dropdowns[i].options = ['No players found']

# Function to handle button click
def on_calculate_button_click(b):
    player_names = [dropdown.value for dropdown in player_dropdowns]
    player_names = [name.strip() for name in player_names if name.strip()]
    
    # Ensure there are exactly 13 players
    if len(player_names) != 13:
        result_label.value = "Please enter exactly 13 players."
        return
    
    avg_points = calculate_average_fantasy_points(player_names)
    if avg_points is not None:
        result_label.value = f"Average fantasy points for the selected players: {avg_points:.2f}"
    else:
        result_label.value = "No valid players found."

# Create widgets for player names with auto-complete dropdowns
player_text_boxes = [widgets.Text(description=f'Player {i+1}:') for i in range(13)]
player_dropdowns = [widgets.Dropdown(options=[]) for _ in range(13)]
calculate_button = widgets.Button(description="Calculate Average")
calculate_button.on_click(on_calculate_button_click)

# Attach the search and update functionality to each text box
for i, text_box in enumerate(player_text_boxes):
    text_box.observe(lambda change, i=i: update_dropdown(change, i), names='value')

result_label = widgets.Label()

# Display widgets with text boxes and dropdowns for each player input
widgets_to_display = [widgets.HBox([text_box, player_dropdowns[i]]) for i, text_box in enumerate(player_text_boxes)]
display(*widgets_to_display, calculate_button, result_label)

Button(description='Calculate Average', style=ButtonStyle())

Label(value='')